In [1]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

dependencies

In [ ]:
!pip install -q git+https://github.com/sunsmarterjie/yolov12.git roboflow supervision flash-attn

example data

In [ ]:
!wget https://media.roboflow.com/notebooks/examples/dog.jpeg

run inference

In [ ]:
import cv2
from ultralytics import YOLO
import supervision as sv


image_path = f"/content/dog.jpeg"
image = cv2.imread(image_path)

model = YOLO('yolov12l.pt')

results = model(image, verbose=False)[0]
detections = sv.Detections.from_ultralytics(results)

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

annotated_image = image.copy()
annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

sv.plot_image(annotated_image)

dataset from Roboflow

Preprocessing

In [ ]:
!pip install imagehash

In [ ]:
import os
import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
import imagehash
from collections import defaultdict

In [ ]:
dataset_path = "/content/drive/MyDrive/Colab Notebooks/AI ML/License Plate Recognition Data Set"
splits = ["train", "valid", "test"]

In [ ]:
# ------------------- 1. Validate annotation pairs -------------------
# -----------------------------------------------------------
# Helper: Plot before/after
def plot_comparison(before_dict, after_dict, title, xlabel, ylabel):
    fig, axs = plt.subplots(1, 2, figsize=(12,5))

    axs[0].bar(before_dict.keys(), before_dict.values(), color="skyblue")
    axs[0].set_title(f"{title} (Before)")
    axs[0].set_xlabel(xlabel); axs[0].set_ylabel(ylabel)

    axs[1].bar(after_dict.keys(), after_dict.values(), color="lightgreen")
    axs[1].set_title(f"{title} (After)")
    axs[1].set_xlabel(xlabel); axs[1].set_ylabel(ylabel)

    plt.show()

# -----------------------------------------------------------
# 1. Validate Annotation Pairs
for split in splits:
    img_dir = Path(dataset_path) / split / "images"
    lbl_dir = Path(dataset_path) / split / "labels"

    img_files = {f.stem for f in img_dir.glob("*")}
    lbl_files = {f.stem for f in lbl_dir.glob("*")}

    before_stats = {"images": len(img_files), "labels": len(lbl_files)}

    missing_labels = img_files - lbl_files
    missing_images = lbl_files - img_files

    for fname in missing_labels:
        (img_dir / f"{fname}.jpg").unlink(missing_ok=True)
        (img_dir / f"{fname}.png").unlink(missing_ok=True)
    for fname in missing_images:
        (lbl_dir / f"{fname}.txt").unlink(missing_ok=True)

    img_files = {f.stem for f in img_dir.glob("*")}
    lbl_files = {f.stem for f in lbl_dir.glob("*")}
    after_stats = {"images": len(img_files), "labels": len(lbl_files)}

    plot_comparison(before_stats, after_stats, f"{split.upper()} - Image/Label Pairs", "Type", "Count")


In [ ]:
# -----------------------------------------------------------
# 2. Image Quality Filtering (delete image + its label)
BLUR_THRESHOLD = 100
CONTRAST_THRESHOLD = 15

for split in splits:
    img_dir = Path(dataset_path) / split / "images"
    lbl_dir = Path(dataset_path) / split / "labels"

    blur_scores = {}
    contrast_scores = {}

    for img_file in img_dir.glob("*"):
        img = cv2.imread(str(img_file), cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        blur_val = cv2.Laplacian(img, cv2.CV_64F).var()
        contrast_val = cv2.calcHist([img],[0],None,[256],[0,256]).std()
        blur_scores[img_file] = blur_val
        contrast_scores[img_file] = contrast_val

    before_stats = {"total_images": len(blur_scores)}

    for img_file, blur_val in blur_scores.items():
        contrast_val = contrast_scores[img_file]

        if blur_val < BLUR_THRESHOLD or contrast_val < CONTRAST_THRESHOLD:
            img_file.unlink(missing_ok=True)

            label_file = lbl_dir / (img_file.stem + ".txt")
            if label_file.exists():
                label_file.unlink()

    after_stats = {"total_images": len(list(img_dir.glob('*')))}

    plot_comparison(before_stats, after_stats, f"{split.upper()} - Image Quality Filter", "Category", "Count")


In [ ]:
# -----------------------------------------------------------
# 3. Check Image File Formats
valid_exts = {".jpg", ".jpeg", ".png"}

for split in splits:
    img_dir = Path(dataset_path) / split / "images"

    # Count before
    before_stats = defaultdict(int)
    for f in img_dir.glob("*"):
        before_stats[f.suffix.lower()] += 1

    # Delete invalid files
    for f in img_dir.glob("*"):
        if f.suffix.lower() not in valid_exts:
            f.unlink()

    # Count after
    after_stats = defaultdict(int)
    for f in img_dir.glob("*"):
        after_stats[f.suffix.lower()] += 1

    plot_comparison(before_stats, after_stats, f"{split.upper()} - File Format Check", "Extension", "Count")



In [ ]:
# -----------------------------------------------------------
# 4. Duplicate Image Detection
for split in splits:
    img_dir = Path(dataset_path) / split / "images"

    seen_hashes = {}
    duplicates = []

    before_count = len(list(img_dir.glob("*")))

    for img_file in img_dir.glob("*"):
        try:
            img = Image.open(img_file)
            h = imagehash.phash(img)
        except:
            continue
        if h in seen_hashes:
            duplicates.append(img_file)
            img_file.unlink()
        else:
            seen_hashes[h] = img_file

    after_count = len(list(img_dir.glob("*")))

    plot_comparison({"images": before_count}, {"images": after_count}, f"{split.upper()} - Duplicate Removal", "Category", "Count")

In [ ]:
dataset_path = "/content/License-Plate-Recognition-11"

!sed -i '$d' {dataset_path}/data.yaml
!sed -i '$d' {dataset_path}/data.yaml
!sed -i '$d' {dataset_path}/data.yaml
!sed -i '$d' {dataset_path}/data.yaml

!echo -e "test: ../test/images\ntrain: ../train/images\nval: ../valid/images" >> {dataset_path}/data.yaml
!cat {dataset_path}/data.yaml


In [ ]:
!cat {dataset_path}/data.yaml

Fine-tunning

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov12s.yaml')

results = model.train(data=f'{dataset_path}/data.yaml', epochs=100)

In [ ]:
from IPython.display import Image

Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=1000)

In [ ]:
from IPython.display import Image

Image(filename=f'{HOME}/runs/detect/train/results.png', width=1000)

In [ ]:
import supervision as sv

ds = sv.DetectionDataset.from_yolo(
    images_directory_path=f"{dataset_path}/test/images",
    annotations_directory_path=f"{dataset_path}/test/labels",
    data_yaml_path=f"{dataset_path}/data.yaml"
)

ds.classes

In [ ]:
from supervision.metrics import MeanAveragePrecision

model = YOLO(f'/{HOME}/runs/detect/train/weights/best.pt')

predictions = []
targets = []

for _, image, target in ds:
    results = model(image, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)

    predictions.append(detections)
    targets.append(target)

map = MeanAveragePrecision().update(predictions, targets).compute()

In [ ]:
print("mAP 50:95", map.map50_95)
print("mAP 50", map.map50)
print("mAP 75", map.map75)

In [ ]:
map.plot()

In [1]:
!pip install supervision

import supervision as sv

model = YOLO(f'/{HOME}/runs/detect/train/weights/best.pt')

ds = sv.DetectionDataset.from_yolo(
    images_directory_path=f"{dataset_path}/test/images",
    annotations_directory_path=f"{dataset_path}/test/labels",
    data_yaml_path=f"{dataset_path}/data.yaml"
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 7.1 MB/s eta 0:00:00


NameError: name 'YOLO' is not defined

In [ ]:
import random

i = random.randint(0, len(ds))

image_path, image, target = ds[i]

results = model(image, verbose=False)[0]
detections = sv.Detections.from_ultralytics(results).with_nms()

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

annotated_image = image.copy()
annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

sv.plot_image(annotated_image)

In [ ]:
from sklearn.metrics import accuracy_score

y_true = []
y_pred = []

for _, image, target in ds:
    results = model(image, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)

    for lbl in target["class_id"]:
        y_true.append(lbl)

    for lbl in detections.class_id:
        y_pred.append(lbl)

min_len = min(len(y_true), len(y_pred))
y_true = y_true[:min_len]
y_pred = y_pred[:min_len]

acc = accuracy_score(y_true, y_pred)
print("Prediction Accuracy:", acc)
